In [1]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *


ImportError: DLL load failed: Le système d’exploitation ne peut pas exécuter %1.

In [3]:
PATH = 'data/'
sz = 100
# architecture used
arch = resnet50
bs = 32

In [4]:
#list of directories in PATH
os.listdir(PATH)

['models', 'tmp', 'Training', 'Validation']

In [5]:
tfms = tfms_from_model(arch, sz, aug_tfms = transforms_side_on, max_zoom = 1.1)
data = ImageClassifierData.from_paths(PATH, bs = bs, tfms=tfms, trn_name="Training", val_name='Validation')

In [6]:
#Give the number of classes
data.c

64

In [7]:
data.classes[:4]

['Apple Braeburn', 'Apple Golden 1', 'Apple Golden 2', 'Apple Golden 3']

Number of pictures the traning set

In [8]:
data.trn_ds.n

31688

In [9]:
#Read in images and their labels given as sub-folder names
learn = ConvLearner.pretrained(arch, data, precompute = True)
lrf = learn.lr_find()

100%|████████████████████████████████████████████████████████████████████████████████| 334/334 [02:15<00:00,  2.47it/s]


 76%|████████████████████████████████████████████████████▎                | 752/991 [00:30<00:09, 25.06it/s, loss=1.27]

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.sched.plot()

In [ ]:
learn = ConvLearner.pretrained(arch, data, precompute = True)
learn.fit(1e-2, 1)

In [ ]:
learn.precompute=False

In [ ]:
learn.fit(1e-2, 3, cycle_len=1)

In [ ]:
learn.unfreeze()

In [ ]:
lr=np.array([1e-4, 1e-3, 1e-2])

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.save('model_resnext50')

In [ ]:
learn.load('model_resnext50')

In [ ]:
log_preds,y = learn.TTA()

probs = np.mean(np.exp(log_preds),0)

In [ ]:
preds = np.argmax(np.mean(log_preds, 0), axis = 1)

In [ ]:
preds = np.argmax(probs, axis = 1)

In [ ]:
accuracy_np(probs, y)

In [ ]:
#Define a function which return n_ex distincts numbers (here means examples) from one mask
def rand_by_mask(mask,n_ex): return np.random.choice(np.where(mask)[0], n_ex, replace=False)

In [ ]:
#This function use previous function and return 4 examples corresponding to correct classfication(correct or wrong)
def rand_by_correct(is_correct, n_ex): return rand_by_mask((preds == data.val_y)==is_correct, n_ex)

In [ ]:
#define a plot function
def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

In [ ]:
def load_img_id(ds, idx): return np.array(PIL.Image.open(PATH+ds.fnames[idx]))

In [ ]:
def plot_val_with_title(idxs, title):
    imgs = [load_img_id(data.val_ds,x) for x in idxs]
    title_probs = [f'{np.max(probs[x],0)}\nPred : {data.classes[np.argmax(probs[x],0)]}\nActual : {data.classes[y[x]]}' for x in idxs]
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8))

In [ ]:
# 1. A few correct labels at random
plot_val_with_title(rand_by_correct(True,4),"Correctly classified")

In [ ]:
# 2. A few incorrect labels at random
plot_val_with_title(rand_by_correct(False,5), "Incorrectly classified")

In [ ]:
def most_by_mask(mask, mult,n_ex):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:n_ex]]

In [ ]:
def most_by_correct(y, is_correct, n_ex):
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask(((preds == data.val_y)==is_correct) & (data.val_y == y), mult, n_ex)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)
confus=[]
for i in range(cm.shape[0]):
    tot=np.sum(cm,1)[i]
    for j in range(cm.shape[0]):
        if cm[i,j]!=0 and cm[i,j]!=tot and data.classes[i]!=data.classes[j]:
            confus.append([cm[i,j], tot, round(cm[i,j]/tot,2), data.classes[i],data.classes[j]])
confus_arr = np.array(confus)

In [ ]:
confus_arr

In [ ]:
cm

In [ ]:
import numpy as np
#np.set_printoptions(threshold=np.nan)

plot_confusion_matrix(cm[0:5,0:5], data.classes[0:5])
